In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split
import sys; sys.path.insert(0,'..')
from main import *
from su_learning import SU_DH as SU
from su_learning import convert_su_data_sklearn_compatible

In [2]:
X_train, y_train = load_svmlight_file('../data/adult/a5a.txt', n_features=123)
X_train = X_train.toarray()

X_test, y_test = load_svmlight_file('../data/adult/a5a.t')
X_test = X_test.toarray()

# SU Compare

In [4]:
n_s = 500
n_u = 500

# Sizes of sets to sample similar, unlabeled points
train_size = 5000
su_cutoff  = 2500

# Only use some of training data
train_samples = np.random.permutation(X_train.shape[0])[:train_size]
X_train = X_train[train_samples]
y_train = y_train[train_samples]

# Sample similar and unlabeled points from disjoint sets
X_s_set = X_train[:su_cutoff]
y_s_set = y_train[:su_cutoff]
X_u_set = X_train[su_cutoff:]
y_u_set = y_train[su_cutoff:]

# Get positive pairs and negative pairs
X_pos = X_s_set[np.where(y_s_set ==  1)]
X_neg = X_s_set[np.where(y_s_set == -1)]

X_pos_idx = np.random.choice(X_pos.shape[0], size=(int(n_s / 2), 2))
X_neg_idx = np.random.choice(X_neg.shape[0], size=(int(n_s / 2), 2))

# Fill in similar pairs
X_s = np.full((n_s, X_train.shape[1] * 2), np.NaN)

k = 0
for (i, j) in X_pos_idx:
    X_s[k] = np.hstack((X_pos[i], X_pos[j]))
    k += 1
    
for (i, j) in X_neg_idx:
    X_s[k] =np.hstack((X_neg[i], X_neg[j]))
    k += 1

# Fill in unlabeled samples
unlabeled_samples = np.random.permutation(X_u_set.shape[0])[:n_u]
X_u = X_u_set[unlabeled_samples]

X_train, y_train = convert_su_data_sklearn_compatible(X_s, X_u)

In [13]:
%%time
clf = SU(prior=0.3, lam=1e-04)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred), (y_pred == 1).all(), (y_pred == -1).all()

CPU times: user 12.3 s, sys: 167 ms, total: 12.5 s
Wall time: 5.43 s


(0.7601254446016752, False, True)